In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Imports + setup drive
import os
import shutil
import sys
import warnings
import datetime
import pytz
warnings.filterwarnings('ignore')

In [3]:
# path utils
BASE_FOLDER_PATH = '/content/drive/MyDrive/Thesis/THOR-Net_based_work'
DATASET_NAME = 'povsurgery' # options: ho3d, povsurgery, TEST_DATASET
PYTHON_VERSION = '3.8'

In [4]:
PATH_TO_HO3D = '/content/drive/MyDrive/Thesis/HO3Dv2'
PATH_TO_POVSURGERY = '/content/drive/MyDrive/Thesis/POV_Surgery_data'
PATH_TO_MANO = '/content/drive/MyDrive/Thesis/mano_v1_2/models'

DATASET_PROCESSING = False
FOLDER_PROCESSED_POVSURGERY = 'object_False'

if DATASET_NAME=='povsurgery':
    PATH_TO_PROCESSED_DATASET = os.path.join(BASE_FOLDER_PATH, DATASET_NAME, FOLDER_PROCESSED_POVSURGERY)
elif DATASET_NAME=='ho3d':
    PATH_TO_PROCESSED_DATASET = os.path.join(BASE_FOLDER_PATH, DATASET_NAME)
else:
    pass

# Install Python version
- __Python 3.7__: Use it to create .npy annotations with **make_data_*.py**
- __Python 3.8__: Use it to train and evaluate the model

In [5]:
if PYTHON_VERSION=='3.7':
    !apt-get update -y
    !apt-get install python3.7 python3.7-distutils
    !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
    !update-alternatives --config python3
    !apt-get install python3-pip
    !python3 -m pip install --upgrade pip --user

    sys.path.append('/usr/local/lib/python3.7/dist-packages')
    sys.executable = '/usr/bin/python3.7'
elif PYTHON_VERSION=='3.8':
    !apt-get update -y
    !apt-get install -y python3.8 python3.8-distutils
    !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
    !update-alternatives --config python3
    !apt-get install python3-pip
    !python3 -m pip install --upgrade pip --user
else:
    pass

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]                
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease         
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                                          
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]                                  
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]             
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]          
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease                
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease         
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:1

In [6]:
print(f'Python version: {os.popen("python --version").read()} (Check it is Python {PYTHON_VERSION}.x)')

Python version: Python 3.8.19
 (Check it is Python 3.8.x)


# Get requirements

In [7]:
!rm -rf /content/sample_data

In [8]:
!pip install --upgrade pip setuptools wheel

  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:0000:0100:01
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.6.0
    Uninstalling setuptools-59.6.0:
      Successfully uninstalled setuptools-59.6.0


In [9]:
# %%capture
# additional requirements
!pip install trimesh
!pip install -U scikit-learn
!pip install chumpy
!pip install ipykernel
!pip install pytz

# requirements from THOR-Net repository
!pip install git+https://github.com/hassony2/manopth.git # It will install manopth==0.0.1
!pip install matplotlib==3.3.4
!pip install numpy==1.13.3
!pip install opencv_python==4.5.3.56
!pip install Pillow==9.2.0
!pip install pymeshlab==2021.10
!pip install scikit_image==0.17.2
!pip install scipy==0.19.1
!pip install skimage==0.0
!pip install torch==1.11.0 # 1.10.0 not working with torchdata
!pip install torchvision==0.12 # 0.11.2 not working torchdata
!pip install torchdata==0.3.0
!pip install tqdm==4.62.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.9/695.9 kB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 25.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 7.3 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 951.6 kB/s eta 0:00:000:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58264 sha256=c401d13e52feadbc278b3467241484a6f7df32b5a68645fa958fd8d008d217bd
  Stored in directory: /root/.cache/pip/wheels/d9/a2/b8/b8aeeeaeb01b5002085156add1aed832f2fb03e79d0f22dfed
Successfully built chumpy
  Using cached packaging-24.1-py3-none-any.whl.metadata 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 3.0 MB/s eta 0:00:00a 0:00:01m
  Cloning https://github.com/hassony2/manopth.git to /tmp/pip-req-build-db7acmh2
  Running command git clone --filter=blob:none --quiet https://github.com/hassony2/manopth.git /tmp/pip-req-build-db7acmh2
  Resolved https://github.com/hassony2/manopth.git to commit 4f1dcad1201ff1bfca6e065a85f0e3456e1aa32b
  Preparing metadata (setup.py) ... done
  Created wheel for manopth: filename=manopth-0.0.1-py3-none-any.whl size=31624 sha256=6b491cf8bf33548c6efc0f09af7ff44efbaa8dda69fd8e1e1e7a1725881d9cef
  Stored in directory: /tmp/pip-ephem-wheel-cache-yqhl8mnz/wheels/09/0d/e5/d1310f393eb55363efca4c9e7dfabcfa2cd4e79d5a228f18f2
Successfully built manopth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.8 MB/s eta 0:00:0000:0100:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 14.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: P

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 791.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 937.1 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 875.2 kB/s eta 0:00:000:00:01


# Set-up dataset and OHRSA-Net

In [10]:
print(f'Python version: {os.popen("python --version").read()} (🟡 WARNING: Check it is Python 3.7.x to run "make_data_*.py")')

Python version: Python 3.8.19
 (🟡 WARNING: Check it is Python 3.7.x to run "make_data_*.py")


In [11]:
# Dataset processing

%cd /content/OHRSA-Net

if DATASET_PROCESSING:
  # WARNING: required Python3.7 for this step
  if not os.path.exists(PATH_TO_PROCESSED_DATASET):
    !mkdir {PATH_TO_PROCESSED_DATASET}
  if DATASET_NAME == 'ho3d':
    !python datasets/make_data_HO3D.py --root {PATH_TO_HO3D} --mano_root {PATH_TO_MANO} --dataset_path {PATH_TO_PROCESSED_DATASET}
  elif DATASET_NAME == 'povsurgery':
    !python datasets/make_data_POV_SURGERY.py --root {PATH_TO_POVSURGERY} --mano_root {PATH_TO_MANO} --dataset_path {PATH_TO_PROCESSED_DATASET}
  else:
    pass # other dataset extension
else:
  # copy *.npy files to 'OHRSA-Net/datasets/{DATASET_NAME}/' folder
  if not os.path.exists(f'datasets/{DATASET_NAME}/'):
    source_path = PATH_TO_PROCESSED_DATASET
    destination_path = f'/content/OHRSA-Net/datasets/{DATASET_NAME}'

    shutil.copytree(source_path, destination_path)

/content/OHRSA-Net


# Training

In [15]:
RESUME_TRAINING_FROM_CHECKPOINT = False

PATH_MODEL_CHECKPOINT_OUTPUT = os.path.join(BASE_FOLDER_PATH, 
    'checkpoints/THOR-Net_trained_on_POV-Surgery_object_False')
MODEL_CHECKPOINT = os.path.join(BASE_FOLDER_PATH, 
    'checkpoints/THOR-Net_trained_on_POV-Surgery_object_False/Training-100samples--20-06-2024_17-08/model-16.pkl')

if RESUME_TRAINING_FROM_CHECKPOINT:
    PATH_MODEL_CHECKPOINT_OUTPUT = os.path.join(MODEL_CHECKPOINT.rpartition(os.sep)[0], 'model-')
else: # New training
    MODEL_CHECKPOINT = ''
    current_timestamp = datetime.datetime.now(pytz.timezone("Europe/Rome")).strftime("%d-%m-%Y_%H-%M")
    folder_output = os.path.join(PATH_MODEL_CHECKPOINT_OUTPUT, f'Training-KE--{current_timestamp}')
    !mkdir {folder_output}
    PATH_MODEL_CHECKPOINT_OUTPUT = os.path.join(folder_output, 'model-')

EPOCHS = 10
BATCH_SIZE = 1

CONNECTIVITY_TYPE = 'base' # hand joints connectivity type

# PATH_TO_OUTPUT_RESULTS = os.path.join(PATH_MODEL_CHECKPOINT_OUTPUT.rpartition(os.sep)[0], 'output_results')

In [13]:
%cd /content/THOR-Net

!export CUDA_DEVICE_ORDER=PCI_BUS_ID
!export CUDA_VISIBLE_DEVICES=0

# command = f'''
!python3.8 main_THOR.py \
  --dataset_name {DATASET_NAME} \
  --root {PATH_TO_PROCESSED_DATASET} \
  --output_file {PATH_MODEL_CHECKPOINT_OUTPUT} \
  --batch_size {BATCH_SIZE} \
  --gpu_number 0 \
  --num_iteration {EPOCHS} \
  --photometric \
  --hid_size 96 \
  --log_batch 1 \
  --val_epoch 1 \
  # --pretrained_model {MODEL_CHECKPOINT} \
  --hands_connectivity_type {CONNECTIVITY_TYPE} \
  # --visualize \
  # --output_results {PATH_TO_OUTPUT_RESULTS} \
  # --object
# os.system(command)

/content/THOR-Net


Traceback (most recent call last):
  File "main_THOR.py", line 9, in <module>
    import torch
  File "/usr/local/lib/python3.8/dist-packages/torch/__init__.py", line 794, in <module>
    import torch.nn.quantizable
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/quantizable/__init__.py", line 1, in <module>
    from .modules import *  # noqa: F403
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/quantizable/modules/__init__.py", line 1, in <module>
    from .activation import MultiheadAttention
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/quantizable/modules/activation.py", line 4, in <module>
    import torch.nn.quantized as nnq
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/quantized/__init__.py", line 1, in <module>
    from .modules import *  # noqa: F403
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/quantized/modules/__init__.py", line 4, in <module>
    from .activation import ReLU6, Hardswish, ELU, LeakyReLU, Sigmoid
  File "/usr/loc

In [22]:
%cd /content/THOR-Net

LOG_FILE = '/content/drive/MyDrive/Thesis/THOR-Net_trained_on_POV-Surgery_object_False/Training-100samples--20-06-2024_17-08/log_Training-100samples--20-06-2024_17-08.txt'
OUTPUT_PATH_PLOTS = '/content/drive/MyDrive/Thesis/THOR-Net_trained_on_POV-Surgery_object_False/Training-100samples--20-06-2024_17-08/out2'

!python3.8 utils/plots.py \
    --log_file {LOG_FILE} \
    --output_path {OUTPUT_PATH_PLOTS}

/content/THOR-Net
Plots saved in "/content/drive/MyDrive/Thesis/THOR-Net_trained_on_POV-Surgery_object_False/Training-100samples--20-06-2024_17-08/out2/Loss_plots--log_Training-100samples--20-06-2024_17-08.png"
Figure(7200x7200)


# Evaluation

In [14]:
%cd /content/THOR-Net

# def copytree(src, dst, copy_function=shutil.copy2):
#     for root, dirs, files in os.walk(src):
#         relative_path = os.path.relpath(root, src)
#         dest_dir = os.path.join(dst, relative_path)
#         os.makedirs(dest_dir, exist_ok=True)
#         for file in files:
#             src_file = os.path.join(root, file)
#             dst_file = os.path.join(dest_dir, file)
#             copy_function(src_file, dst_file)

# # copy 'checkpoints'folder from drive to './checkpoints' folder under 'content/THOR-Net'
# source_path = '/content/drive/MyDrive/Thesis/THOR-Net_based_work/checkpoints'
# destination_path = '/content/THOR-Net/checkpoints'

# copytree(source_path, destination_path)

/content/THOR-Net


In [15]:
%%capture
# Needed for requirements incompatibility
!pip uninstall -y numpy
!pip install numpy==1.19.5

In [16]:
CHECKPOINT_MODEL = '/content/drive/MyDrive/Thesis/THOR-Net_based_work/checkpoints/THOR-Net_trained_on_HO3D/model-18.pkl'
BASE_PATH_TO_OUTPUTS_RESULTS = '/content/drive/MyDrive/Thesis/THOR-Net_based_work'
PATH_TO_OUTPUT_RESULTS = os.path.join(BASE_PATH_TO_OUTPUTS_RESULTS, 
    'output_results')
CHECKPOINT_ID = 18 # not used
CONNECTIVITY_TYPE = 'base' # hand joints connectivity type
GPU_NUMBER = 0

In [18]:
!export CUDA_DEVICE_ORDER=PCI_BUS_ID
!export CUDA_VISIBLE_DEVICES=0
PATH_TO_book_obj = '/content/THOR-Net/datasets/objects/mesh_1000/book.obj'

!python3.8 test_THOR.py \
--testing \
--dataset_name {DATASET_NAME} \
--root {PATH_TO_PROCESSED_DATASET} \
--checkpoint_model {CHECKPOINT_MODEL} \
--output_results {PATH_TO_OUTPUT_RESULTS} \
--mano_root {PATH_TO_MANO} \
--obj_root {PATH_TO_book_obj} \
--split test \
--seq SM1 \
--seq rgb \
--gpu_number {GPU_NUMBER} \
--batch_size 1 \
--hid_size 96 \
--photometric \
--hands_connectivity_type {CONNECTIVITY_TYPE} \
--object \
# --visualize


# %cd /content/THOR-Net
# os.system('cd /content/THOR-Net')
# os.system(command)

args:
testing: True | dataset_name: ho3d | mano_root: /content/drive/MyDrive/Thesis/mano_v1_2/models | output_results: /content/drive/MyDrive/Thesis/THOR-Net_based_work/output_results | checkpoint_model: /content/drive/MyDrive/Thesis/THOR-Net_based_work/checkpoints/THOR-Net_trained_on_HO3D/model-18.pkl | hands_connectivity_type: base | obj_root: /content/THOR-Net/datasets/objects/mesh_1000/book.obj | root: /content/drive/MyDrive/Thesis/THOR-Net_based_work/ho3d | adj_matrix_root: /datasets/adj_matrix | output_file: ./checkpoints/model- | pretrained_model:  | hdf5_path:  | batch_size: 1 | gpu_number: 0 | learning_rate: 0.0001 | lr_step: 100 | lr_step_gamma: 0.9 | log_batch: 100 | val_epoch: 1 | snapshot_epoch: 1 | num_iterations: 50 | object: True | num_features: 2048 | hid_size: 96 | freeze: False | buffer_size: 1000 | photometric: True | seq:  | checkpoint_folder: ho | checkpoint_id: 1 | visualize: False | split: test | 

Loading evaluation data ... ✅ Evaluation data loaded.
🟢 Model "T